Diagnosing Student Code with Avicenna
===

In this Jupyter Notebook, we are exploring an interesting problem in educational technology and computer science education: the automated evaluation and diagnosis of student-submitted code. While automated systems can easily evaluate the correctness of code, diagnosing the _reasons_ behind any incorrect behavior can be a complex task.

### The Problem: Greatest Common Divisor (GCD)

Students have been assigned the task of writing a Python function to calculate the Greatest Common Divisor (GCD) of two integers. The GCD of two integers is the largest integer that divides both numbers without a remainder.

### Avicenna: Our Debugging and Diagnosis Tool

To aid us in this endeavor, we will be using Avicenna, an automated debugging tool that uses fuzz testing and symbolic reasoning to identify the conditions under which code behaves incorrectly. Avicenna's goal is to automatically diagnose software faults with high precision and recall, thereby making the debugging process more efficient.

With this notebook, we aim to:

1. Showcase how Avicenna can be used to diagnose student code.
2. Illustrate the capabilities of Avicenna through an example of diagnosing a GCD implementation.
3. Demonstrate how the diagnosis output can be used for generating new failure-inducing inputs.

By the end of this notebook, you'll have a clearer understanding of how automated diagnosis can aid in educational settings, and how tools like Avicenna contribute to this.

### Student Implementation

Below is the student's GCD implementation in Python.

In [1]:
# Student's implementation for calculating GCD
class Solution:
    def gcd(self, A, B):
        l=[]
        if A >= B:
            for i in range(1, A):
                if A % i == 0 and B % i == 0:
                    l.append(i)
            return l[-1] if len(l) > 1 else l[0]
        else:
            for i in range(1, B):
                if A % i == 0 and B % i == 0:
                    l.append(i)
            return l[-1] if len(l) > 1 else l[0]

### Reference Implementation

This is a reference implementation for calculating the GCD using the Euclidean algorithm.

In [2]:
# Reference implementation using the Euclidean algorithm
class Reference():
    def gcd(self, A, B):
        if A < B:
            A, B = B, A
        while B:
            A, B = B, A % B
        return A

### Testing the Implementations

Let's run some initial tests to compare the student and reference implementations.

In [3]:
# Comparing the two implementations
ob = Solution()
print(ob.gcd(10, 10))  # Output should be 10 for correct implementation

ref = Reference()
print(ref.gcd(10, 10))  # Output should be 10 as well

5
10


### Fuzzing Setup

Here we set up a grammar for fuzz testing using GrammarFuzzer.

In [4]:
# Setting up a grammar for fuzzing
from fuzzingbook.Grammars import Grammar
import string 

grammar: Grammar = {
    '<start>': ["<input>"],
    "<input>": ["<first> <second>"],
    "<first>": ["<integer>"],
    "<second>": ["<integer>"],
    "<integer>": ["<onenine><maybe_digits>"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
}

### Oracle Construction

We will now use the reference implementation to construct an oracle for fuzz testing.

In [5]:
# Constructing an oracle using the reference GCD implementation
from evogfuzz.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

error_def = {UnexpectedResultError: OracleResult.BUG}
oracle = construct_oracle(ob.gcd, ref.gcd, error_def)

### Running the Fuzzer

Let's run the fuzzer and examine the results.

In [6]:
# Running the fuzzer to test the student's implementation
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

fuzzer = GrammarFuzzer(grammar)
for _ in range(10):
    inp = fuzzer.fuzz()
    print(inp.ljust(30), oracle(inp))

455 873                        NO_BUG
7564 69922                     NO_BUG
3 56                           NO_BUG
5 4                            NO_BUG
34 764                         NO_BUG
962439 5                       NO_BUG
8 7                            NO_BUG
7 9                            NO_BUG
95 68                          NO_BUG
6 71                           NO_BUG


### EvoGGen

In [7]:
# Using Avicenna for diagnosis
from evogfuzz.evogfuzz_class import EvoGGen

initial_inputs = ["10 2", "4 4", "49 6618904968"]

evoggen = EvoGGen(
    grammar=grammar,
    inputs=initial_inputs,
    oracle=oracle
)

In [9]:
grammar, found_exception_inputs = evoggen.optimize()

In [10]:
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 11 bug-triggering inputs!


In [11]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

7 7
5 5
2 2
61 61
8 8
9 9
4 4
6 6
433 433
4 4
3 3


In [14]:
for rule in grammar:
    print(rule.ljust(40), grammar[rule])

<start>                                  [('<input>', {'prob': None})]
<input>                                  [('<first> <second>', {'prob': None})]
<first>                                  [('<integer>', {'prob': None})]
<second>                                 [('<integer>', {'prob': None})]
<integer>                                [('<onenine><maybe_digits>', {'prob': None})]
<onenine>                                [('1', {'prob': 0.0}), ('2', {'prob': 0.09090909090909091}), ('3', {'prob': 0.09090909090909091}), ('4', {'prob': 0.2727272727272727}), ('5', {'prob': 0.09090909090909091}), ('6', {'prob': 0.18181818181818182}), ('7', {'prob': 0.09090909090909091}), ('8', {'prob': 0.09090909090909091}), ('9', {'prob': 0.09090909090909091})]
<digit>                                  [('0', {'prob': 0.0}), ('1', {'prob': 0.3333333333333333}), ('2', {'prob': 0.0}), ('3', {'prob': 0.6666666666666666}), ('4', {'prob': 0.0}), ('5', {'prob': 0.0}), ('6', {'prob': 0.0}), ('7', {'prob': 0.0}), (

In [28]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer

prob_fuzzer = ProbabilisticGrammarFuzzer(grammar)

for _ in range(20):
    inp = prob_fuzzer.fuzz()
    print(inp.ljust(20), oracle(inp))

7 9                  NO_BUG
431 4                NO_BUG
7 7                  BUG
8 43                 NO_BUG
3 73                 NO_BUG
4 6                  NO_BUG
41 3                 NO_BUG
4 4                  BUG
7 9                  NO_BUG
7 6                  NO_BUG
7 61                 NO_BUG
63 63                BUG
3 613                NO_BUG
9 4                  NO_BUG
4 4                  BUG
4 3                  NO_BUG
71 4                 NO_BUG
7 3                  NO_BUG
73 6                 NO_BUG
5 6                  NO_BUG


In [29]:
## Timeout

In [31]:
# Constructing an oracle using the reference GCD implementation
from evogfuzz.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

error_def = {TimeoutError: OracleResult.BUG}
oracle = construct_oracle(ob.gcd, ref.gcd, error_def)

In [41]:
from evogfuzz.evogfuzz_class import EvoGGen

initial_inputs = ["10 2", "4 4", "49 6618904968"]

evoggen = EvoGGen(
    grammar=grammar,
    inputs=initial_inputs,
    oracle=oracle
)

In [42]:
grammar, found_exception_inputs = evoggen.optimize()

In [43]:
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 45 bug-triggering inputs!


In [44]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

72 594902591
49 6618904968
5 774897396
23448 270198924
8 59781891
7 88166159
3 60858386
53 41030889577995208
43543501 9
5 18741753083
87885353 57
9 56073491
44266959679041 38082885
9712941144 95
31621862667288386 7
4 24096601114
37 41421780
95 4722743555
40611170 20499
5 83310843


In [47]:
from fuzzingbook.ProbabilisticGrammarFuzzer import ProbabilisticGrammarFuzzer
from evogfuzz.input import Input
prob_fuzzer = ProbabilisticGrammarFuzzer(grammar)

eval_data = []
for _ in range(100):
    inp = prob_fuzzer.fuzz()
    inp_ = Input.from_str(grammar, inp, oracle=oracle(inp))
    eval_data.append(inp_)
    #print(inp.ljust(20), oracle(inp))
    print(str(inp_).ljust(40), inp_.oracle)

739366437328 4534                        BUG
9 8                                      NO_BUG
4 6670                                   NO_BUG
5 79689768                               BUG
5 2395                                   NO_BUG
2933675872056995901545 29578870          BUG
22754 16701                              NO_BUG
659 13951798915234                       BUG
8604298038296416 495443                  BUG
3 8                                      NO_BUG
27 44452792423                           BUG
9 7                                      NO_BUG
9 206466280404                           BUG
621827 82                                NO_BUG
4488131986263610 868469412506501         BUG
549 332638300388349007                   BUG
1 7                                      NO_BUG
913083304 398361                         BUG
2601464 59215802012                      BUG
996916 7990161994805420034982            BUG
7 4                                      NO_BUG
54286757448273 5045015      

In [48]:

bug_list = [inp for inp in eval_data if inp.oracle == OracleResult.BUG] 

In [49]:
len(bug_list)

45

In [5]:
## EvoGFuzz

In [45]:
# Constructing an oracle using the reference GCD implementation
from evogfuzz.oracle_construction import construct_oracle, UnexpectedResultError, OracleResult

err = {UnexpectedResultError: OracleResult.BUG, TimeoutError: OracleResult.UNDEF}
oracle = construct_oracle(ob.gcd, ref.gcd, err)

In [46]:
# initial_inputs = ["10 2", "4 4", "49 6618904968"]
initial_inputs = ["10 2", "15 20"]

In [47]:
for inp in initial_inputs:
    print(inp.ljust(30), oracle(inp))

10 2                           NO_BUG
15 20                          NO_BUG


In [48]:
from evogfuzz import EvoGFuzz

evogfuzz = EvoGFuzz(
    grammar=grammar,
    inputs=initial_inputs,
    oracle=oracle
)

In [49]:
found_exception_inputs = evogfuzz.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 8 bug-triggering inputs!


In [50]:
for inp in list(found_exception_inputs)[:250]:
    print(str(inp).ljust(30), inp.oracle)

20 20                          BUG
17 17                          BUG
28 28                          BUG
29 29                          BUG
2 2                            BUG
26 26                          BUG
10 10                          BUG
25 25                          BUG


In [51]:
evogfuzz.report

Report for EvoGFuzz
Found 8 failure-inducing inputs (1 Exceptions):
None: 8